In [ ]:
using Pkg

Pkg.activate("/home/rs/.julia/environments/triangulations/")

In [ ]:
using Random
Random.seed!(1);

# CairoMakie plotting basics

In [ ]:
using AbstractPlotting
using CairoMakie

In [ ]:
n = 7

x = 400 * rand(n)
y = 200 * rand(n)
[x y]'

In [ ]:
scene = Scene(resolution=(400,200), show_axis=false, scale_plot=false)
scatter!(x, y, markersize=5)
linesegments!(x, y)

In [ ]:
make_scene() = Scene(resolution=(400,200), show_axis=false, scale_plot=false)

# TriangleMesh.jl

In [ ]:
using Statistics

In [ ]:
using TriangleMesh

In [ ]:
# simple point cloud meshing
p = [x y]
mesh = create_mesh(p);

In [ ]:
mesh.point

In [ ]:
mesh.edge

In [ ]:
mesh.segment

In [ ]:
edge_array = mesh.point[:, vec(mesh.edge)]

In [ ]:
"Draw mesh based on initial points."
function draw_mesh(mesh, points)
    scene = make_scene()
    linesegments!(mesh.point[:, vec(mesh.edge)]', color=:gray, linestyle=:dot)
    linesegments!(mesh.point[:, vec(mesh.segment)]')
    scatter!(points, markersize=8, color=:red)
    scatter!(mesh.point', markersize=4)
end

In [ ]:
draw_mesh(mesh, p)

In [ ]:
# again, but no Steiner points
mesh = create_mesh(p, prevent_steiner_points=true, delaunay=true)
draw_mesh(mesh, p)

In [ ]:
mesh1 = mesh;

In [ ]:
mesh1.segment

In [ ]:
mesh1.edge

In [ ]:
# refine this mesh
mesh = refine(mesh1, keep_edges=true)
draw_mesh(mesh, p)

In [ ]:
# refine this mesh
mesh = refine(mesh1)
draw_mesh(mesh, p)

In [ ]:
# refine this mesh
mesh = refine_rg(mesh1)
draw_mesh(mesh, p)

In [ ]:
# easy to understand and repeatable, but still awkward Steiner points?
mesh2 = mesh;

In [ ]:
# refine this mesh, again
mesh = refine_rg(mesh2)
draw_mesh(mesh, p)

In [ ]:
# looks too regular :-\ try again

In [ ]:
# refine this mesh, again
mesh = refine(mesh2, divide_cell_into=1, keep_segments=true)
draw_mesh(mesh, p)

In [ ]:
# just using pre-existing meshing methods and refinements is not good enough :-(

In [ ]:
mesh1.edge

In [ ]:
function polygon_from_mesh(mesh)
    points = mesh1.point
    segments = mesh1.edge

    poly = Polygon_pslg(size(points, 2), 0, 0, size(segments, 2), 0)
    set_polygon_point!(poly, points')
    set_polygon_segment!(poly, segments')

    return poly
end

In [ ]:
poly1 = polygon_from_mesh(mesh1)

In [ ]:
mesh1 = create_mesh(poly1, add_switches="S0")
draw_mesh(mesh1, p)

In [ ]:
function add_points(mesh, points)
    points = hcat(mesh.point, points)
    segments = mesh.segment

    poly = Polygon_pslg(size(points, 2), 0, 0, size(segments, 2), 0)
    set_polygon_point!(poly, points')
    set_polygon_segment!(poly, segments')

    # no extra Steiner points
    return create_mesh(poly, add_switches="S0")
end

In [ ]:
mesh1.point

In [ ]:
mesh1.cell

In [ ]:
triangle_centers(mesh) = mean(mesh.point[:, mesh.cell], dims=2)[:,1,:]

In [ ]:
mesh2 = add_points(mesh1, triangle_centers(mesh1))
draw_mesh(mesh2, p)

In [ ]:
mesh3 = add_points(mesh2, triangle_centers(mesh2))
draw_mesh(mesh3, p)

# Triangle.jl

In [ ]:
using Triangle

In [ ]:
# Start with straight-forward Delaunay triangulation, again.
points = [x y]
points_map = collect(1:n)

triangles = basic_triangulation(points, points_map)
@show triangles

edges = reshape(vcat([t[[1, 2, 2, 3, 3, 1]] for t in triangles]...), 2, :)
@show edges

scene = Scene(resolution=(600,300))
scatter!(x, y, markersize=8, color=:red)
linesegments!(points[vec(edges), :], linewidth=2)

In [ ]:
# for each triangle, add a point in the center (mean)
centers = [mean(points[t, :], dims=1) for t in triangles]

In [ ]:
centers_array = reshape(hcat(centers...), 2, :)'
@show centers_array
scatter!(centers_array, markersize=5)

In [ ]:
edges'[:,:]

In [ ]:
points2 = vcat(points, centers_array)
points_map2 = collect(1:size(points2, 1))
triangles2 = constrained_triangulation(points2, points_map2, edges'[:,:])

edges2 = reshape(vcat([t[[1, 2, 2, 3, 3, 1]] for t in triangles2]...), 2, :)
linesegments!(points2[vec(edges2), :], color=:green)

In [ ]:
# need also subdivide the edges that we want to keep?
# TODO...